In [1]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

c:\Users\Sourav Chaudhary\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
data = pd.read_csv("ssn_multiclass_data.csv")

In [3]:
data.head()

,text,label
0,This is a general message without any personal...,No SSN
1,Please send me the documents tomorrow.,No SSN
2,The user's SSN is 854-23-9935.,Contains SSN
3,The user's SSN is 532-14-1488.,Contains SSN
4,The user's SSN is 338-74-1434.,Contains SSN


In [4]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data.text, data.label, test_size=0.2, random_state=42)

In [6]:
Y_train[:5]

5894    1
3728    1
8958    2
7671    1
5999    2
Name: label, dtype: int64

In [7]:
X_train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
X_test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)

In [8]:
X_train_encodings['input_ids']

[[101,
  2292,
  1005,
  1055,
  4608,
  2039,
  2279,
  2733,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [101,
  14764,
  1024,
  2136,
  3116,
  2012,
  1017,
  7610,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [101,
  7591,
  2951,
  1024,
  7020,
  2078,
  2003,
  22038,
  2595,
  1011,
  22038,
  1011,
  4347,
  1012,
  102,
  0,
  0,
  0],
 [101,
  2023,
  2003,
  1037,
  2236,
  4471,
  2302,
  2151,
  3167,
  8909,
  4765,
  28295,
  1012,
  102,
  0,
  0,
  0,
  0],
 [101,
  7591,
  2951,
  1024,
  7020,
  2078,
  2003,
  22038,
  2595,
  1011,
  22038,
  1011,
  6191,
  14526,
  1012,
  102,
  0,
  0],
 [101,
  2023,
  2003,
  1037,
  2236,
  4471,
  2302,
  2151,
  3167,
  8909,
  4765,
  28295,
  1012,
  102,
  0,
  0,
  0,
  0],
 [101,
  1996,
  5310,
  1005,
  1055,
  7020,
  2078,
  2003,
  4805,
  2620,
  1011,
  2676,
  1011,
  4185,
  16703,
  1012,
  102,
  0],
 [101,
  14764,
  1024,
  2136,
  3116,
  2012,
  1017,
  7610,
  1012,
  102,
 

In [9]:
X_train_encodings[:5]

[Encoding(num_tokens=18, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=18, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=18, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=18, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=18, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [10]:
X_train

5894                             Let's catch up next week.
3728                       Reminder: team meeting at 3 PM.
8958                   Sensitive data: SSN is XXX-XX-1937.
7671     This is a general message without any personal...
5999                   Sensitive data: SSN is XXX-XX-6311.
                               ...                        
11284                       The user's SSN is 560-66-9729.
11964         The quick brown fox jumps over the lazy dog.
5390                        The user's SSN is 329-98-4587.
860                         The user's SSN is 518-33-3218.
15795                  Sensitive data: SSN is XXX-XX-8272.
Name: text, Length: 16000, dtype: object

In [11]:
import tensorflow as tf

In [28]:
X_train_tensor = tf.data.Dataset.from_tensor_slices((dict(X_train_encodings), Y_train.tolist())).batch(32)
X_test_tensor = tf.data.Dataset.from_tensor_slices((dict(X_test_encodings), Y_test.tolist())).batch(32)

In [29]:
from transformers import TFDistilBertForSequenceClassification, TFTrainingArguments
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [30]:
# ✅ Compile
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# ✅ Train
model.fit(X_train_tensor, validation_data=X_test_tensor, epochs=3)

Epoch 1/3



500/500 [==============================] - 437s 833ms/step - loss: 0.0212 - accuracy: 0.9957 - val_loss: 1.6147e-04 - val_accuracy: 1.0000
Epoch 2/3
500/500 [==============================] - 324s 648ms/step - loss: 1.0381e-04 - accuracy: 1.0000 - val_loss: 3.0498e-05 - val_accuracy: 1.0000
Epoch 3/3
500/500 [==============================] - 313s 625ms/step - loss: 3.0759e-05 - accuracy: 1.0000 - val_loss: 1.1084e-05 - val_accuracy: 1.0000


In [31]:
model.save_pretrained("./distilbert-ssn-tf")
tokenizer.save_pretrained("./distilbert-ssn-tf")

('./distilbert-ssn-tf\\tokenizer_config.json',
 './distilbert-ssn-tf\\special_tokens_map.json',
 './distilbert-ssn-tf\\vocab.txt',
 './distilbert-ssn-tf\\added_tokens.json',
 './distilbert-ssn-tf\\tokenizer.json')

In [32]:
from sklearn.metrics import classification_report, accuracy_score
Y_pred = model.predict(X_test_tensor)

125/125 [==============================] - 17s 130ms/step


In [33]:
print("Accuracy:", accuracy_score(Y_test, Y_pred.logits.argmax(axis=1)))
print(classification_report(Y_test, Y_pred.logits.argmax(axis=1), target_names=label_encoder.classes_))

Accuracy: 1.0
              precision    recall  f1-score   support

Contains SSN       1.00      1.00      1.00      1209
      No SSN       1.00      1.00      1.00      1973
 Partial SSN       1.00      1.00      1.00       818

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



In [34]:
from sklearn.metrics  import confusion_matrix
print(confusion_matrix(Y_test, Y_pred.logits.argmax(axis=1)))



[[1209    0    0]
 [   0 1973    0]
 [   0    0  818]]


In [35]:
# Load model and tokenizer
model = TFDistilBertForSequenceClassification.from_pretrained("./distilbert-ssn-tf")
tokenizer = DistilBertTokenizerFast.from_pretrained("./distilbert-ssn-tf")

# Label mapping
id2label = {0: 'Contains SSN', 1: 'Partial SSN', 2: 'No SSN'}

Some layers from the model checkpoint at ./distilbert-ssn-tf were not used when initializing TFDistilBertForSequenceClassification: ['dropout_79']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./distilbert-ssn-tf and are newly initialized: ['dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
def predict_ssn_class(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True)
    
    # Run prediction
    outputs = model(inputs)
    logits = outputs.logits

    # Get predicted class index
    predicted_class_id = int(tf.argmax(logits, axis=1).numpy()[0])
    
    # Convert to label
    return id2label[predicted_class_id]

In [38]:
text = "My SSN is 123-45-6789"
print(predict_ssn_class(text))  # Example output: Contains SSN

Contains SSN
